<a href="https://colab.research.google.com/github/aditya-saurabh/Coursera_Capstone/blob/master/Totonto_neighborhood_battle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import requests
import lxml.html as lh

In [0]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [0]:
page = requests.get(url)

In [0]:
doc = lh.fromstring(page.content)

In [0]:
tr_elements = doc.xpath('//tr')

In [6]:
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content().split("\n")[0]
    print (i,name)
    col.append((name,[]))

1 Postcode
2 Borough
3 Neighbourhood


In [0]:
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 10, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content()
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [0]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)
df['Neighbourhood'] = df['Neighbourhood'].str.rstrip("\n")

In [9]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [0]:
# Filtering neighbourhood
for i in range(df.shape[0]):
  if df.iloc[i]['Neighbourhood'] == 'Not assigned':
    df.iloc[i]['Neighbourhood'] = df.iloc[i]['Borough']

### This has been done to assign neighbourhood the same borough if it is 'Not assigned' ###

### Now rows where Boroughs are not assigned are dropped ###

In [0]:
df = df[df['Borough'] != 'Not assigned']
df.reset_index(inplace = True, drop = True)

In [12]:
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


In [13]:
df_filtered = df
df_filtered.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


In [14]:
df_filtered['Neighbourhood'].describe()

count           210
unique          207
top       Runnymede
freq              2
Name: Neighbourhood, dtype: object

In [0]:
df_filtered['Neighbourhood'] = df_filtered['Neighbourhood'] + ', '

In [16]:
df_grouped = df_filtered[['Postcode','Neighbourhood']].groupby(by='Postcode').sum()
df_grouped

,Neighbourhood
Postcode,
M1B,"Rouge, Malvern,"
M1C,"Highland Creek, Rouge Hill, Port Union,"
M1E,"Guildwood, Morningside, West Hill,"
M1G,"Woburn,"
M1H,"Cedarbrae,"
...,...
M9N,"Weston,"
M9P,"Westmount,"
M9R,"Kingsview Village, Martin Grove Gardens, Richv..."


In [0]:
df_grouped['Neighbourhood'] = df_grouped['Neighbourhood'].str.rstrip(', ')

In [0]:
df_grouped.reset_index(inplace=True, drop = False)

In [19]:
df_grouped.head()

,Postcode,Neighbourhood
0,M1B,"Rouge, Malvern"
1,M1C,"Highland Creek, Rouge Hill, Port Union"
2,M1E,"Guildwood, Morningside, West Hill"
3,M1G,Woburn
4,M1H,Cedarbrae


In [0]:
df_test = df_grouped

In [0]:
d = {}
for key, value in zip(df['Postcode'], df['Borough']):
  d[key] = value

In [0]:
d

In [0]:
df_test['Borough'] = df_test['Postcode'].map(d)

# This is the final dataset #

In [24]:
df_test.head()

,Postcode,Neighbourhood,Borough
0,M1B,"Rouge, Malvern",Scarborough
1,M1C,"Highland Creek, Rouge Hill, Port Union",Scarborough
2,M1E,"Guildwood, Morningside, West Hill",Scarborough
3,M1G,Woburn,Scarborough
4,M1H,Cedarbrae,Scarborough


In [25]:
df_test.describe()

,Postcode,Neighbourhood,Borough
count,103,103,103
unique,103,102,11
top,M5M,Queen's Park,North York
freq,1,2,24


In [26]:
df_test.shape

(103, 3)